In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [3]:
load_dotenv(override=True)

True

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

This function generates output word by word - Just like in ChatGPT

In [6]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end = "", flush = True)

Subject: Simplify Your SOC 2 Compliance Process with AI

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am reaching out to introduce you to ComplAI, a cutting-edge SaaS solution designed to streamline SOC 2 compliance and audit preparation.

In today’s regulatory landscape, ensuring compliance can be both time-consuming and complex. ComplAI harnesses the power of AI to automate and simplify the entire compliance process, enabling organizations like yours to save valuable time and resources while maintaining the highest standards of security and trust.

Key benefits of our solution include:

- **Automated Documentation:** Generate necessary compliance documentation with ease.
- **Real-Time Monitoring:** Stay updated on your compliance status and potential risks.
- **Audit Readiness:** Prepare for audits proactively, ensuring a smooth review process.

I would love to discuss how ComplAI can specifically address your organization’s needs and en

In [7]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance Journey with ComplAI

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ComplAI, a cutting-edge SaaS solution designed to streamline SOC 2 compliance and audit preparation through the power of AI.

In today's regulatory landscape, ensuring compliance can be daunting and time-consuming. Many organizations face challenges such as inefficient manual processes, lack of visibility into compliance requirements, and the ever-increasing pressure to maintain standards. ComplAI addresses these issues head-on, enabling companies like yours to not only meet compliance requirements but also to do so with confidence and ease.

Key features of ComplAI include:

- **Automated Risk Assessments**: Effortlessly evaluate potential risks with our AI-driven tools.
- **Real-Time Reporting**: Gain instant visibility into your compliance status with comprehensive dashboards.
- **Audit Preparation**: Simplify the audit pro

In [8]:
# Picks the best email out of all the generated ones 

sales_picker = Agent(
    name = "sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [9]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

    outputs = [result.final_output for result in results]
    emails = "Cold Sales Email: \n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)
    print(f"Best Sales Email: \n\n {best.final_output}")

Best Sales Email: 

 Subject: Is Your SOC2 Compliance as Relaxed as Your Last Vacation?

Hey [First Name],

Hope you’re not too swamped diving into your pile of paper audits—oh wait, you shouldn’t be! 😅 

At ComplAI, we believe SOC2 compliance should feel less like a root canal and more like a stroll through the park. Our AI-powered tool takes the guesswork out of audits, helping you chill out while we handle the nitty-gritty details. Think of us as your compliance sidekick—like Robin to your Batman, but with fewer capes and more spreadsheets.

Curious how we can turn your audit prep into a walk in the park? Let’s chat! I promise, it’ll be more fun than your last team-building exercise.

Looking forward to hearing from you!

Best,  
[Your Name]  
[Your Position]  
ComplAI  
P.S. If you’re still using sticky notes for compliance, it’s time for an upgrade! Just sayin’. 😉


Making use of Tools

In [10]:
sales_agent1 = Agent(
    name = "Professional Sales Agent",
    instructions= instructions1,
    model = "gpt-4o-mini"
)
sales_agent2 = Agent(
    name = "Engaging Sales Agent",
    instructions= instructions2,
    model = "gpt-4o-mini"
)
sales_agent3 = Agent(
    name = "Busy Sales Agent",
    instructions= instructions3,
    model = "gpt-4o-mini"
)

In [11]:
sales_agent1

Agent(name='Professional Sales Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

Steps 2,3 - Agent and Tool interactions

In [12]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("uditsdaily@gmail.com")  # Change to your verified sender
    to_email = To("uditdgreat@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [13]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015EFC8F6440>, strict_json_schema=True, is_enabled=True)

Converting an Agent into a Tool

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015EFCFC9630>, strict_json_schema=True, is_enabled=True)

Gather all the tools together

A tool for each of our agents 

and a tool for our function to send emails

In [18]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015EFCFCB520>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015EFCFCAD40>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

In [19]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across (Delegation)

In [20]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name = "Email Subject Writer", instructions=subject_instructions, model = "gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name = "HTML email body converter", instructions=html_instructions, model = "gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name = "html_converter", tool_description="Convert a text email into a HTML email body")

In [21]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email('uditsdaily@gmail.com')
    to_email = To('uditdgreat@gmail.com')
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body = mail)
    return {"status" : "success"}

In [22]:
tools = [subject_tool, html_tool, send_html_email]

In [23]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015EFCFCB6D0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email into a HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000015EFD1E4040>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to all 

In [24]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."

emailer_agent = Agent(
    name = "Email Manager",
    instructions=instructions,
    tools = tools, 
    model = "gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it"
)

Now in total, we have 3 tools and 1 Handoff

In [25]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [28]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name = "Sales Manager",
    instructions=sales_manager_instructions,
    tools = tools,
    handoffs = handoffs, 
    model = "gpt-4o-mini"
)

message = "Send out a cold email addressed to Dear CEO from Udit"

with trace("Automated SDR - 3 Tools, 1 handoff"):
    result = await Runner.run(sales_manager, message)